# **IFM Tag Calibration Script**

This Jupyter notebook was created to calculate and deliver metrics about thermal characteristics of IFM's T-sensing Fibers for the purposes of calibrating and tesing their performance prior to delivery.

#### **How to use:**

In the Cell below, fill in the capitalized variables. TAGS is a list of last-four-digit EPCs to process, and DATA_DIREC is the folder where data is located. The folder named in DATA_DIREC should be filled with individual folders containing two files, one being the tags.csv and the other being the RSSI.csv.

Then, run every cell, and the output files will be in a folder in the same directory as this notebook, titled by the variable OUTPUT_NAME. 

In [ ]:
# List of strings of LAST FOUR DIGITS of tag EPCs to be used in calculations.
TAGS = [
    '2B42', '3943', '3242'
]

# Name of the folder where outputted files will be placed.
OUTPUT_NAME = 'Tag Results'

# Name of the folder where
DATA_DIREC = 'DATA'

In [ ]:
# NECESSARY IMPORTS FOR SCRIPT TO RUN
# If error occurs in this cell, open Terminal/Command Prompt and pip install "package name"
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime, os
import matplotlib.dates as mdates
import pandas as pd
from sklearn.metrics import r2_score
import pandas as pd
from tqdm import tqdm


# HELPER FUNCTIONS FOR LOADING DATA
# These should not cause errors--if they do, let me know.

def MakeDataFromText(filepath:str):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif line.count(":")==2:
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h), int(m), int(s)))
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db

def MakeDict(data:list):
    """
    """
    t1234 = dict()
    for d in data:
        for k,v in d.items():
            if k in t1234: # if tag is already in dict
                for i in range(len(v[0])):
                    if v[0][i] in t1234[k]:
                        t1234[k][v[0][i]].append(v[1][i])
                    else:
                        t1234[k][v[0][i]] = [v[1][i]]
            else:             # if tag is not in dict
                t1234[k] = dict()
                for i in range(len(v[0])):
                    t1234[k][v[0][i]] = [v[1][i]]
    for k,v in t1234.items():
        for k2,v2 in v.items():
            t1234[k][k2] = sum(v2)/len(v2)
    return t1234

In [ ]:
############################################################################################################################
# DO NOT CHANGE CODE IN THIS SECTION, FOLLOW ERROR MESSAGE #################################################################
############################################################################################################################

os.mkdir(OUTPUT_NAME) # Create output folder if it does not exist

DATA_NAMES, TEMP_LIST, RSSI_LIST = [], [], []
for D__ in os.listdir(DATA_DIREC):
    DATA_NAMES.append(D__)         # List of all data file names in DATA_DIREC

    path = os.path.join(DATA_DIREC, D__)
    t,r = dict(), dict()
    for prefix in os.listdir(path):
        filepath = os.path.join(path, prefix)
        if   prefix[:4]=='tags':
            TEMP_LIST.append( MakeDataFromText(filepath) ) # Create a temperature dictionary (hash table)
        elif prefix[:4]=='RSSI':
            RSSI_LIST.append( MakeDataFromText(filepath) ) # Create an RSSI dictionary (hash table)

    # print(len(TEMP_LIST), len(RSSI_LIST))
    assert len(TEMP_LIST)==len(RSSI_LIST)==len(DATA_NAMES), "Format Error: {} does not contain a tags_NUM.csv and RSSI_NUM.csv".format(path)

TEMP_LIST, RSSI_LIST, DATA_NAMES = np.array(TEMP_LIST), np.array(RSSI_LIST), np.array(DATA_NAMES) # NP arrays for speed
############################################################################################################################

### **Excel Code**

This section of the notebook creates a file called "tags_coefficients.xlsx", in Excel sheet format, in the directory OUTPUT_NAME chosen at the start of the notebook file. This Excel has three sheets, one for All RSSIs, one for RSSI<=20, and RSSI>20. It includes data for when RSSI is exactly equal to ten. Here is a list of the action items it fulfills:

In [ ]:
with pd.ExcelWriter(OUTPUT_NAME + "/tags_coefficients_.xlsx") as writer:

    for SHEETS in range(3):  # Three Sheets, for All RSSIs, RSSI<=20, and RSSI>20
        list_dfs = []
        for TAG in TAGS:
            r2_l, x2_l, x_l, c_l, r2_q, x2_q, x_q, c_q  = [], [], [], [], [], [], [], []

            for u in range(len(TEMP_LIST)):
                for k,v in TEMP_LIST[u].items():
                    if k[-4:]==TAG:
                        st, sr = sorted( TEMP_LIST[u][k].items() ), sorted( RSSI_LIST[u][k].items() )
                        dt, tt, dr = [ j[1] for j in st ], [ j[0] for j in st ], [ j[1] for j in sr ]

            